In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from pathlib import Path
from pprint import pprint
import urltools
import numpy as np
import csv

import matplotlib.pyplot as plt 
import seaborn as sns

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

import configparser

from _helpers import *

In [2]:
pd.set_option("display.precision", 2)
sns.set_style("darkgrid")

In [17]:
root = Path('../../')
Config = configparser.ConfigParser()
Config.read(str(root / 'config.cnf'))

# Load files from disk
queries = root / Config.get('input_files', 'queries')
queries = load_queries(str(queries))

am = root / Config.get('input_files', 'altmetric')
am = pd.read_csv(str(am))
# am_ = pd.read_excel("../../data/input/NewsMentions-AltmetricOct2017.xlsx", sheet_name=1)

tweets = root / Config.get('output_files', 'tweets')

twitter_urls = root / Config.get('output_files', 'twitter_urls')
twitter_urls = load_urls(twitter_urls)

summary_tweets = root / Config.get('output_files', 'summary_tweets')
summary_tweets = pd.read_csv(summary_tweets)
summary_tweets.set_index("venue", inplace=True)

summary_urls = root / Config.get('output_files', 'summary_urls')

## Errors

In [5]:
exp = pd.read_csv("temp/expanded_urls.csv", index_col=0, parse_dates=['timestamp'])

/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Work with final dataset

In [18]:
urls.sample(10)

,error,expanded,relevant_url,resolved_url,short_url,timestamp,tweet_id,venue
id,,,,,,,,
59748,NaN,NaN,NaN,http://www.latimes.com/science/sciencenow/la-s...,http://lat.ms/2tR6Exe,2018-09-21 05:51:43.639179,NaN,expanded_urls.csv
264583,NaN,True,https://www.nytimes.com/2017/02/02/science/sle...,NaN,NaN,2018-11-22 13:10:20.405515,827322873371033600,nytimes
126343,NaN,False,https://www.nytimes.com/2016/12/04/science/bre...,NaN,NaN,2018-11-22 07:35:58.897542,805644656549707648,nytimes
213383,NaN,True,https://www.washingtonpost.com/news/energy-env...,NaN,NaN,2018-11-23 16:57:42.026611,824732922502664320,washingtonpost
148284,NaN,True,https://www.washingtonpost.com/news/speaking-o...,NaN,NaN,2018-12-24 12:51:10.638293,883390779623841920,washingtonpost
501792,NaN,NaN,NaN,https://www.washingtonpost.com/news/to-your-he...,https://t.co/LVERrtU1yy,2018-09-26 18:25:57.111907,NaN,expanded_urls.csv
241803,NaN,NaN,NaN,https://www.iflscience.com:443/plants-and-anim...,http://fb.me/Pa0G5p0p,2018-09-23 11:23:28.231520,NaN,expanded_urls.csv
289668,NaN,NaN,NaN,https://www.iflscience.com:443/space/how-to-ba...,http://fb.me/Z9tWid7c,2018-09-23 21:19:17.646914,NaN,expanded_urls.csv
75277,NaN,False,http://www.latimes.com/science/sciencenow/la-s...,NaN,NaN,2018-11-22 03:13:15.562692,890913617826074624,latimes


In [12]:
urls.to_csv("twitter_urls.csv")

In [13]:
summary_tweets['found tweets'].sum(), urls.relevant_url.count(), urls.cleaned_url.nunique(), 100*urls.relevant.sum()/summary_tweets['found tweets'].sum()

(3584984, 1607182, 18062, 44.71735438707676)

In [14]:
summary_tweets['found urls'] = urls.venue.value_counts()
summary_tweets['relevant urls'] = urls.groupby("venue")['relevant'].sum()
summary_tweets['unique urls'] = urls.groupby("venue")['cleaned_url'].nunique()

x = summary_tweets[['found tweets', 'found urls', 'relevant urls', 'unique urls']]

x[['relevant urls']] = x[['relevant urls']].div(x['found tweets']/100, axis="rows")

x.columns = ['found tweets', 'found urls', 'relevant urls [%]', 'unique urls']
x = x.round(1).replace(np.nan, 0)
x.to_csv(summary_urls)
x

/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,found tweets,found urls,relevant urls [%],unique urls
venue,,,,
bostonglobe,16468,4182.0,25.4,226.0
chicago,830,0.0,0.0,0.0
foxnews,229911,149869.0,65.2,3475.0
iflscience,184242,0.0,0.0,0.0
latimes,125809,84034.0,66.8,2066.0
nytimes,1124602,686997.0,60.9,4569.0
sfchronicle,9863,4984.0,50.5,325.0
slate,1873,646.0,34.5,219.0
theglobeandmail,26130,4897.0,18.7,176.0


In [84]:
summary_tweets

,found tweets,original,retweets,quotes,retweets/quotes,reply,found urls,relevant urls,unique urls
venue,,,,,,,,,
bostonglobe,16468,3996,11616,317,539,129,4182.0,4182.0,226.0
chicago,830,265,564,1,0,4,NaN,NaN,NaN
foxnews,229911,100453,88014,2032,39412,2363,149869.0,149868.0,3475.0
iflscience,184242,128366,53984,1488,404,2947,NaN,NaN,NaN
latimes,125809,53174,71081,514,1040,1088,84034.0,84031.0,2066.0
nytimes,1124602,422636,687368,5432,9166,6588,686997.0,684332.0,4569.0
sfchronicle,9863,2822,6970,39,32,40,4984.0,4984.0,325.0
slate,1873,876,942,15,40,21,646.0,646.0,219.0
theglobeandmail,26130,8034,17999,45,52,583,4897.0,4897.0,176.0
